This notebook takes the raw data found in multiple files within the "raw" folder and places them all into one file for a given month. The data is separated into motor vehicle theft and theft and finally geocoded so that it can be used in the map construction.

In [1]:
import pandas as pd
import os
import geocoder
import requests
import time

In [2]:
time.sleep(15000)
for folder in os.listdir('./raw'):
    file_list = []
    for file in os.listdir('./raw/' + folder):
        if(file.endswith('.xls')):
            data = pd.read_excel('./raw/' + folder + '/' + file)
            file_list.append(data)
   
    df = pd.concat(file_list, ignore_index=True)
    
    columns = ['Incident Number','Date', 'Time', 'Police District','Offense 1',
          'Offense 2', 'Offense 3', 'Offense 4', 'Offense 5', 'Location']
    df.columns = columns
    
    df = df[df['Offense 1'].notnull()]
    df = df[df['Location'].notnull()]
    
    df.to_csv('./raw/' + folder + 'Complete.csv', sep=';', index = False)

In [3]:
def address_to_latlong(address):
    '''Returns the latitude and longitude for a given address'''
    address += ", Milwaukee, WI"
    location = geocoder.arcgis(address, session = session)
    
    if not location.latlng:
        print("Couldn't Parse: "  + address )
    else:
        print(f'Parsed: {address} - {location.latlng}')
    return location.latlng

In [4]:
for file in os.listdir('./raw/'):
    if(file.endswith('.csv')):
        df = pd.read_csv('./raw/'+ file, sep =';')
    
        zipped = zip(df['Offense 1'].items(), df['Offense 2'].items(), 
        df['Offense 3'].items(), df['Offense 4'].items(), df['Offense 5'].items())
        keyword = 'theft'
        
        for off_1, off_2, off_3, off_4, off_5 in zipped:
            if(not keyword in off_1[1].lower() and
              (pd.isnull(off_2[1]) or not keyword in off_2[1].lower()) and
              (pd.isnull(off_3[1]) or not keyword in off_3[1].lower()) and
              (pd.isnull(off_4[1]) or not keyword in off_4[1].lower()) and
              (pd.isnull(off_5[1]) or not keyword in off_5[1].lower())):
                df = df.drop(off_1[0])
 
        df['Address'] = df['Location'].copy()
            
        theft = df.copy()
        m_theft = df.copy()
        
        zipped = zip(df['Offense 1'].items(), df['Offense 2'].items(), 
        df['Offense 3'].items(), df['Offense 4'].items(), df['Offense 5'].items())
        keyword = 'motor vehicle theft'
        
        for off_1, off_2, off_3, off_4, off_5 in zipped:
            if(not keyword in off_1[1].lower() and
                (pd.isnull(off_2[1]) or not keyword in off_2[1].lower()) and
                (pd.isnull(off_3[1]) or not keyword in off_3[1].lower()) and
                (pd.isnull(off_4[1]) or not keyword in off_4[1].lower()) and
                (pd.isnull(off_5[1]) or not keyword in off_5[1].lower())):
                m_theft = m_theft.drop(off_1[0])
            else:
                theft = theft.drop(off_1[0])
                
        
        with requests.Session() as session:
            # This is likely to be very slow
            # depending on the geocoding service used
            theft['Location'] = theft['Location'].apply(address_to_latlong)
            m_theft['Location'] = m_theft['Location'].apply(address_to_latlong)
            
        theft.to_csv('./' + 'theft_' + file.split('Complete.csv')[0] + '.csv', sep=';', index = False)
        m_theft.to_csv('./' + 'm_theft_' + file.split('Complete.csv')[0] + '.csv', sep=';', index = False)
        print(f'Finished {file}')

Parsed: 2032 W EGGERT PL, Milwaukee, WI - [43.11086132444748, -87.93826790235582]
Parsed: 4572 N 23RD ST, Milwaukee, WI - [43.10048632944776, -87.94049641185313]
Parsed: 4146 N 13TH ST, Milwaukee, WI - [43.09243082653356, -87.92692538631792]
Parsed: 1931 W FIEBRANTZ AV, Milwaukee, WI - [43.091411481245466, -87.93536441909515]
Parsed: 4800 N GREEN BAY AV, Milwaukee, WI - [43.10438121270349, -87.9311260175431]
Parsed: 4136 N 10TH ST, Milwaukee, WI - [43.092442561198, -87.92322638631792]
Parsed: 4800 N TEUTONIA AV, Milwaukee, WI - [43.10463706996437, -87.94658937901069]
Parsed: 5036 N 28TH ST, Milwaukee, WI - [43.1089639103526, -87.94772235356932]
Parsed: 7343 N TEUTONIA AV, Milwaukee, WI - [43.15132464778698, -87.95769584142163]
Parsed: 4032 N 25TH ST, Milwaukee, WI - [43.090578220093505, -87.94461540463978]
Parsed: 6151 N TEUTONIA AV, Milwaukee, WI - [43.12895717620306, -87.95224264186731]
Parsed: 4678 N 19TH ST, Milwaukee, WI - [43.10215401165675, -87.93409546292358]
Parsed: 4614 N 29T

Parsed: 920 N HAWLEY RD, Milwaukee, WI - [43.04083771412088, -87.98290034176833]
Parsed: 2567 N 51ST ST, Milwaukee, WI - [43.065728444630395, -87.97750062479058]
Parsed: 2728 N 54TH ST, Milwaukee, WI - [43.06869816180969, -87.98092441185312]
Parsed: 5815 W APPLETON AV, Milwaukee, WI - [43.06839760357952, -87.98561553290538]
Parsed: 6901 W VIENNA AV, Milwaukee, WI - [43.08601798024103, -87.99815857607416]
Parsed: 4131 W MARTIN DR, Milwaukee, WI - [43.04589454284749, -87.9664620693923]
Parsed: 5707 W LISBON AV, Milwaukee, WI - [43.06653245294027, -87.98450383071714]
Parsed: 1542 N 51ST ST, Milwaukee, WI - [43.05111274854292, -87.9775253974264]
Parsed: 1503 N 50TH PL, Milwaukee, WI - [43.05031199999999, -87.97675860257358]
Parsed: 2808 N 51ST ST #REAR, Milwaukee, WI - [43.069968387731564, -87.97739442238472]
Parsed: 1907 N 48TH ST, Milwaukee, WI - [43.055424000000016, -87.97357759925535]
Parsed: 4033 W ST PAUL AV, Milwaukee, WI - [43.03475425392439, -87.96447371403042]
Parsed: 5219 W CENT

Parsed: 616 W VIRGINIA ST, Milwaukee, WI - [43.02643216591921, -87.91856504039802]
Parsed: 212 S BARCLAY ST, Milwaukee, WI - [43.029223268191906, -87.90960862508474]
Parsed: 983 W LINCOLN AV, Milwaukee, WI - [43.00282748845882, -87.924672]
Parsed: 2206 S 15TH ST, Milwaukee, WI - [43.00452124588953, -87.9310819100171]
Parsed: 155 S 1ST ST, Milwaukee, WI - [43.029909902014055, -87.91104148185103]
Parsed: 1208 S 17TH ST, Milwaukee, WI - [43.01890417242347, -87.93424546501585]
Parsed: 1222 S 8TH ST, Milwaukee, WI - [43.018506, -87.92107642296163]
Parsed: 1001 W SCOTT ST, Milwaukee, WI - [43.019027466818734, -87.92411221870852]
Parsed: 2527 S 10TH ST, Milwaukee, WI - [42.99858092200935, -87.9249400226497]
Parsed: 224 W WASHINGTON ST, Milwaukee, WI - [43.02014345325739, -87.91323202553522]
Parsed: 1576 W HOWARD AV, Milwaukee, WI - [42.973904460470756, -87.93358158090484]
Parsed: 1102 W MALLORY AV, Milwaukee, WI - [42.94660286246047, -87.92728125886141]
Parsed: 2016 W HOWARD AV, Milwaukee, WI

Parsed: 1503 N 38TH ST, Milwaukee, WI - [43.050258000000014, -87.9607981392173]
Parsed: 2620 W LLOYD ST, Milwaukee, WI - [43.057809511541194, -87.94703266763806]
Parsed: 3132 W LLOYD ST, Milwaukee, WI - [43.05787950043269, -87.95345064210284]
Parsed: 2808 N 38TH ST, Milwaukee, WI - [43.06996816763808, -87.96089385356932]
Parsed: 2325 N 50TH ST, Milwaukee, WI - [43.06127329808413, -87.97621901543633]
Parsed: 2402 N 22ND ST, Milwaukee, WI - [43.06240830391252, -87.94000638963614]
Parsed: 2220 W LLOYD ST, Milwaukee, WI - [43.057824205781195, -87.94038517280927]
Parsed: 2204 N 34TH ST, Milwaukee, WI - [43.059438, -87.95628338631792]
Parsed: 2672 N 41ST ST, Milwaukee, WI - [43.0677, -87.9644934190665]
Parsed: 2508 N 10TH ST, Milwaukee, WI - [43.06425266472388, -87.92399640463978]
Parsed: 2242 N 12TH ST, Milwaukee, WI - [43.060076413266785, -87.92685587520941]
Parsed: 2813 N 33RD ST, Milwaukee, WI - [43.07008516763807, -87.95516961368207]
Parsed: 2661 N 45TH ST, Milwaukee, WI - [43.067430586

Parsed: 5776 N 94TH ST, Milwaukee, WI - [43.122977413266796, -88.02942286078269]
Parsed: 4367 N 60TH ST, Milwaukee, WI - [43.09669444653092, -87.98693389469476]
Parsed: 4250 N 68TH ST, Milwaukee, WI - [43.09473541326679, -87.99691336799604]
Parsed: 6200 W FOND DU LAC AV, Milwaukee, WI - [43.10023112171303, -87.98915393137135]
Parsed: 4458 N 75TH ST, Milwaukee, WI - [43.09852575437128, -88.00559438631792]
Parsed: 5400 W CAPITOL DR, Milwaukee, WI - [43.09006224678438, -87.98077074678436]
Parsed: 5940 N 67TH ST, Milwaukee, WI - [43.12584866472389, -87.9945953280341]
Parsed: 4411 N 60TH ST, Milwaukee, WI - [43.097514173750625, -87.98691934750124]
Parsed: 3933 N 51ST BL, Milwaukee, WI - [43.08876836081134, -87.97711061757722]
Parsed: 4157 N 73RD ST, Milwaukee, WI - [43.09304383236193, -88.00305110646872]
Parsed: 5255 N 68TH ST, Milwaukee, WI - [43.11342041909515, -87.99607811757723]
Parsed: 4535 N 74TH ST, Milwaukee, WI - [43.099901580904856, -88.00450813200395]
Parsed: 7230 W CARMEN AV, Mi

...

Parsed: 1112 S 23RD ST #A, Milwaukee, WI - [43.01973000000001, -87.94217545181507]
Parsed: 1203 S 33RD ST, Milwaukee, WI - [43.018895711983475, -87.95518768713514]
Parsed: 3016 S 39TH ST, Milwaukee, WI - [42.989823, -87.96291489353128]
Parsed: 1228 S 36TH ST, Milwaukee, WI - [43.018327089647414, -87.95880593738836]
Parsed: 2310 W FOREST HOME AV, Milwaukee, WI - [43.004301317242735, -87.94259558840666]
Parsed: 3130 W GREENFIELD AV, Milwaukee, WI - [43.0169315115412, -87.95353066763806]
Parsed: 1260 S 36TH ST, Milwaukee, WI - [43.017670089647424, -87.95878598845881]
Parsed: 1037 S 28TH ST, Milwaukee, WI - [43.020135167638074, -87.94911060257358]
Parsed: 2117 W PIERCE ST #LOWER, Milwaukee, WI - [43.024170470136944, -87.94008]
Parsed: 2100 W PIERCE ST, Milwaukee, WI - [43.02421951659018, -87.93946043060768]
Parsed: 3014 W PIERCE ST, Milwaukee, WI - [43.02388942108573, -87.95200545943247]
Parsed: 1558 S 35TH ST, Milwaukee, WI - [43.014599161809684, -87.9575634190665]
Parsed: 2164 S MUSKEGO 

Parsed: 7101 W APPLETON AV, Milwaukee, WI - [43.0851521869442, -88.00045092926973]
Parsed: 2215 N 51ST ST, Milwaukee, WI - [43.059506878045745, -87.97759239722637]
Parsed: 5000 W BURLEIGH ST, Milwaukee, WI - [43.07536522808298, -87.97660372808298]
Parsed: 2844 N 61ST ST, Milwaukee, WI - [43.070886, -87.9885448607827]
Parsed: 601 N 54TH ST, Milwaukee, WI - [43.03773292339432, -87.98103005539828]
Parsed: 1638 N 60TH ST, Milwaukee, WI - [43.0520938332152, -87.98749315204141]
Parsed: 5615 W MARTIN DR, Milwaukee, WI - [43.04605094751592, -87.98354689246341]
Parsed: 2663 N 56TH ST, Milwaukee, WI - [43.06742103136361, -87.98337758425171]
Parsed: 3903 N 67TH ST, Milwaukee, WI - [43.08822100582839, -87.9958880809335]
Parsed: 224 S 61ST ST #LWR, Milwaukee, WI - [43.02897300000001, -87.98866538631792]
Parsed: 3835 N 70TH ST, Milwaukee, WI - [43.086856642506866, -87.99873838331682]
Parsed: 2701 N 51ST ST, Milwaukee, WI - [43.06804650573104, -87.97752667714471]
Parsed: 3281 N 58TH ST, Milwaukee, WI

Parsed: 2026 N 30TH ST, Milwaukee, WI - [43.05706149708581, -87.95093288631791]
Parsed: 1919 N 24TH ST, Milwaukee, WI - [43.055226502914195, -87.9427106281088]
Parsed: 2557 N 10TH ST, Milwaukee, WI - [43.0652874446304, -87.92406063589908]
Parsed: 2616 N 9TH ST, Milwaukee, WI - [43.066232664723884, -87.92260335356933]
Parsed: 2641 N 33RD ST, Milwaukee, WI - [43.06692831556927, -87.95520007703836]
Parsed: 2300 N 16TH ST, Milwaukee, WI - [43.06050246653746, -87.93256527802615]
Parsed: 3722 W BROWN ST, Milwaukee, WI - [43.05647753318129, -87.96069]
Parsed: 3700 W GALENA ST, Milwaukee, WI - [43.05154650764606, -87.960141]
Parsed: 2125 N 47TH ST, Milwaukee, WI - [43.057638974464766, -87.97241759536023]
Parsed: 3739 W VLIET ST, Milwaukee, WI - [43.04867680564227, -87.96085082697981]
Parsed: 2700 N 10TH ST, Milwaukee, WI - [43.06769190852041, -87.92396426747521]
Parsed: 2608 N 46TH ST, Milwaukee, WI - [43.06633183236195, -87.97117239353128]
Parsed: 2406 N 38TH ST, Milwaukee, WI - [43.062659832

Parsed: 2919 W WISCONSIN AV, Milwaukee, WI - [43.038635502885526, -87.95097033527612]
Parsed: 1334 N 22ND ST, Milwaukee, WI - [43.048089000000004, -87.9402159190665]
Parsed: 2020 W STATE ST, Milwaukee, WI - [43.043272478792595, -87.93842302553523]
Parsed: 1815 W MC KINLEY AV, Milwaukee, WI - [43.04719553952926, -87.93585067055226]
Parsed: 2801 W STATE ST, Milwaukee, WI - [43.04320951399404, -87.9491612514571]
Parsed: 820 N JACKSON ST, Milwaukee, WI - [43.04204820734396, -87.90429858679457]
Parsed: 945 N 12TH ST, Milwaukee, WI - [43.04278832207815, -87.92721784805246]
Parsed: 200 N 35TH ST, Milwaukee, WI - [43.03295562423784, -87.95768515184623]
Parsed: 2900 W HIGHLAND BL, Milwaukee, WI - [43.044640457152525, -87.95089341618096]
Parsed: 7911 W HAMPTON AV, Milwaukee, WI - [43.10491548456368, -88.010892]
Parsed: 8652 W GRANTOSA DR, Milwaukee, WI - [43.10243412075123, -88.02161534667309]
Parsed: 8909 W STARK ST, Milwaukee, WI - [43.10674851788917, -88.0237125]
Parsed: 8909 W STARK ST, Milw

Parsed: 3525 N 36TH ST, Milwaukee, WI - [43.08312675437128, -87.95847763200395]
Parsed: 3020 N 39TH ST, Milwaukee, WI - [43.07376566472388, -87.96205541517136]
Parsed: 4302 W CAPITOL DR, Milwaukee, WI - [43.08991348600597, -87.96758047446477]
Parsed: 2450 W NASH ST, Milwaukee, WI - [43.08473514111008, -87.94370890426796]
Parsed: 3041 N 45TH ST, Milwaukee, WI - [43.07434233527613, -87.96989960646873]
Parsed: 3271 N 30TH ST, Milwaukee, WI - [43.07852767055226, -87.95094913200396]
Parsed: 4779 N HOPKINS ST, Milwaukee, WI - [43.104369719949275, -87.95987644145639]
Parsed: 3429 N 40TH ST, Milwaukee, WI - [43.08166900582839, -87.96318855150314]
Parsed: 3019 N 29TH ST, Milwaukee, WI - [43.073829754371275, -87.94980163200395]
Parsed: 3626 W FOND DU LAC AV, Milwaukee, WI - [43.07692945158396, -87.95944045848019]
Parsed: 3632 W BURLEIGH ST, Milwaukee, WI - [43.07523548211083, -87.95939416763807]
Parsed: 3055 N 27TH ST, Milwaukee, WI - [43.074612586733224, -87.94729115032581]
Parsed: 2976 N 27TH 